## Теоретическая часть

---


1. Ответьте на вопросы:  
В чем принципиальное отличие гибридных рекомендательных систем от коллаборативной филтьтрации?  
Приведите 2-3 примера задач, в которых необходимо использовать гибридные системы  




В коллаборативной фильтрации используются только матрицы интеракций, рекомендации строятся на основе похожих пользователей. В гибридных системах к матрице интеракций добавляются внешние признаки, что в комбинации улучшает рекомендацию

* Ленты новостей в соцсетях 
* Онлайн-магазины
* Онлайн-кинотеатры

---

2.  Прочитайте статью про поиск на hh.ru https://habr.com/ru/company/hh/blog/347276/
Нам интересна именно рекомендательная система, раздел "Производительность системы" можно пропустить
Какие основные отличия предложенной системы от тех подходов, которые мы разбирали на семинарах? Какие проблемы могут возникнуть при выводе такой модели в продакшен?

Используют две модели.
* Первая - линейная, используется для того, чтобы быстро и с малой ресурсоёмкостью отделять подходящие вакансии от неподходящих и грубо ранжировать неподходящие. 
* Вторая - XGboost, используется, чтобы более точно ранжировать подходящие

---

3. На вебинаре мы рассматривали модель LightFM (https://making.lyst.com/lightfm/docs/lightfm.html). В работе Data Scientist'а важную часть занимает research - исследование существующих архитектур и разбор научных статей, в которых они описываются. Вам предлагается изчуть оригинальную статью про LightFM https://arxiv.org/pdf/1507.08439.pdf и ответить на следующие вопросы:

1) Какой датасет используют авторы?

* MovieLens 10M dataset в сочетании с набором тегов Tag Genome
* CrossValidated

2) Что используют в качестве признаков?

* Каждому фильму относится список жанров и список тэгов. У каждого тэга стоит рейтинг точности от 0 до 1, описывающий, как точно тэг подходит фильму. Также у каждого фильма стоит рейтинг из оценок всех пользователей.
* Вопросы и ответы пользователей

3) С какими моделями сравнивают LightFM? Опишите их основные идеи кратко

1. MF: обычная матричная модель факторизации.
2. LSI-LR: логистическая регрессияю. Основывается на факторизации матрицы контента.
3. LSI-UP: гибридная модель, которая представляет профили пользователей, как линейную комбинацию предметов по векторам, затем применяет LSI к получившейся матрице, чтобы получить скрытые представления пользователей и элементов.
4. LightFM (теги): модель LightFM, использующая только функции тегов.
5. LightFM (теги + идентификаторы): модель LightFM, использующая функции тегов и индикаторов элементов.
6. LightFM (теги + информация): модель LightFM, использующая как предметные, так и пользовательские функции.

---

## Практическая часть


---

In [ ]:
import lightfm

C:\ProgramData\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:10: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  "LightFM was compiled without OpenMP support. "


In [ ]:
# import data
# utils functions like in webinar

### 1. Модуль SRC

На вебинаре было рассказано про модуль src. Он приложен в материалах. Скачайте его, изучите структуру, импортируйте функции

### 2. Работа с признаками

У нас есть внешние данные. Что с ними не так? Чего не хватает?  

Проведите исследование внешних данных и составьте какие-нибудь содержательные выводы.  
Формально Вам нужно построить 3+ графиков (scatter plot, hist или что-то иное) и описать, что мы видим (например, товары такой-то категории болле часто покупаются в следующие дни недели или пользователи с большим достатком предпочитают такие-то товары).  
Исследуйте те закономерности, которые Вам интересно, чем менее тривиальный вывод получается, тем лучше! 

### 3. LightFM

У этого алогритма есть множество параметров (item/user_alpha, loss, no_components).  
Проведите эксперименты аналогично дз 3 (подберите гипперпараметры каким удобно способои и постройте графики)  
На выходе необходимо получить pr@5 на валидации (последние 3 недели) > 2%  

У Вас, скорее всего, возникнет проблема со временем обучения. Почему они возникает?    

Попробуйте запустить алгоритм вообще без фичей или используйте только признаки с небольшим числом уникальных категорий. (item_features['commodity_desc'].unique() - 300 уникальных категорий - это очень много)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.sparse import csr_matrix, coo_matrix

from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
data = pd.read_csv('../webinar_2/retail_train.csv')
item_features = pd.read_csv('../webinar_2/product.csv')
user_features = pd.read_csv('../webinar_2/hh_demographic.csv')

item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
k=5000

top_k = data_train.groupby('item_id')['quantity'].sum().reset_index()\
    .sort_values('quantity', ascending=False).head(k).item_id.tolist()

data_train.loc[~data_train['item_id'].isin(top_k), 'item_id'] = 999999

/home/sergey/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [4]:
# Отношение кол-ва походов в магазин покупателя за товаром 
# к кол-ву купленных товаров

user_item_matrix = pd.pivot_table(
    data_train, 
    index='user_id', columns='item_id', 
    values=['basket_id', 'quantity'],
    aggfunc={
        'basket_id': 'count',
        'quantity': 'count'
    },
    fill_value=0
)

user_item_matrix = user_item_matrix.basket_id / user_item_matrix.quantity
user_item_matrix.fillna(0, inplace=True)

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15778533,15831255,15926712,15926775,15926844,15926886,15927403,15927661,15927850,16809471
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]

In [6]:
test_user_item_matrix = pd.pivot_table(
    data_test,
    index='user_id', columns='item_id',
    values=['basket_id', 'quantity'],
    aggfunc={
        'basket_id': 'count',
        'quantity': 'count'},
    fill_value=0
)

test_user_item_matrix = test_user_item_matrix.basket_id / test_user_item_matrix.quantity
test_user_item_matrix.fillna(0, inplace=True)
sparse_user_item_test = csr_matrix(test_user_item_matrix).tocsr()

In [7]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [8]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)

item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

In [9]:
user_feat_lightfm = pd.get_dummies(
    user_feat, 
    columns=user_feat.columns.tolist()
)

item_feat_lightfm = pd.get_dummies(
    item_feat, 
    columns=item_feat.columns.tolist()
)

In [20]:
no_components_list = [5, 10, 15]
loss_list=['warp', 'bpr']
learning_rate_list=[0.01, 0.05, 0.1]
item_alpha_list=[0.05, 0.1, 0.15]
user_alpha_list=[0.05, 0.1, 0.15]

models = []

# k = len(no_components_list) * len(loss_list) * len(learning_rate_list)\
#     * len(item_alpha_list) * len(user_alpha_list)

user_item_matrix_0_1 = (sparse_user_item > 0) * 1
sample_weight = coo_matrix(user_item_matrix)
user_features = csr_matrix(user_feat_lightfm.values).tocsr()
item_features = csr_matrix(item_feat_lightfm.values).tocsr()

for no_components in no_components_list:
    for loss in loss_list:
        for learning_rate in learning_rate_list:
            for item_alpha in item_alpha_list:
                for user_alpha in user_alpha_list:
                    
#                     print(k, end=', ')
                    
                    model = LightFM(
                        no_components=no_components,   # кол-во компонент (какой размерностью будут эмбединги по фичам)
                        loss=loss,
                        learning_rate=learning_rate, # шаг для обучения
                        item_alpha=item_alpha,     # смещение по товару
                        user_alpha=user_alpha,     # смещение по пользователю
                        random_state=42
                    )
            
                    model.fit(
                        user_item_matrix_0_1, # user-item matrix из 0 и 1
                        sample_weight=sample_weight,
                        user_features=user_features,
                        item_features=item_features,
                        epochs=15
                    )
                
                    models.append(model)
#                     k-=1

162, 161, 160, 159, 158, 157, 156, 155, 154, 153, 152, 151, 150, 149, 148, 147, 146, 145, 144, 143, 142, 141, 140, 139, 138, 137, 136, 135, 134, 133, 132, 131, 130, 129, 128, 127, 126, 125, 124, 123, 122, 121, 120, 119, 118, 117, 116, 115, 114, 113, 112, 111, 110, 109, 108, 107, 106, 105, 104, 103, 102, 101, 100, 99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 

In [27]:
max_precision = 0
sp = []
# k = len(models)

for model in models:
    
#     print(k, end=', ')
    
    train_precision = precision_at_k(
        model, 
        sparse_user_item,
        user_features=user_features,
        item_features=item_features,
        k=5
        ).mean()
            
    test_precision = precision_at_k(
        model, 
        sparse_user_item_test,
        user_features=user_features,
        item_features=item_features,
        k=5
    ).mean()
            
    sp.append([
        round(train_precision, 4), round(test_precision, 4),
        model.no_components, model.loss, model.learning_rate,
        model.item_alpha, model.user_alpha
    ])
            
    if test_precision > max_precision:
                
        max_precision = test_precision
                
        best_no_components = no_components
        best_loss = loss
        best_learning_rate = learning_rate
        best_item_alpha = item_alpha
        best_user_alpha = user_alpha
        
#     k-=1
    
best_params = {
    'no_components': best_no_components,
    'loss': best_loss, 
    'learning_rate': best_learning_rate,
    'item_alpha': best_item_alpha,
    'user_alpha': best_user_alpha
}

162, 161, 160, 159, 158, 157, 156, 155, 154, 153, 152, 151, 150, 149, 148, 147, 146, 145, 144, 143, 142, 141, 140, 139, 138, 137, 136, 135, 134, 133, 132, 131, 130, 129, 128, 127, 126, 125, 124, 123, 122, 121, 120, 119, 118, 117, 116, 115, 114, 113, 112, 111, 110, 109, 108, 107, 106, 105, 104, 103, 102, 101, 100, 99, 98, 97, 96, 95, 94, 93, 92, 91, 90, 89, 88, 87, 86, 85, 84, 83, 82, 81, 80, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 68, 67, 66, 65, 64, 63, 62, 61, 60, 59, 58, 57, 56, 55, 54, 53, 52, 51, 50, 49, 48, 47, 46, 45, 44, 43, 42, 41, 40, 39, 38, 37, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 26, 25, 24, 23, 22, 21, 20, 19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 

In [53]:
print('  Максимальное значение метрики precision_at_k достигается при следующих параметрах:\n')
for i, j in best_params.items(): print(f'{i:<7}:\t{j:>4}')
print(f'\n  И имееет значение:\n\nmax_precision_@k = {max_precision}')

  Максимальное значение метрики precision_at_k достигается при следующих параметрах:

no_components:	  15
loss   :	 bpr
learning_rate:	 0.1
item_alpha:	0.15
user_alpha:	0.15

  И имееет значение:

max_precision_@k = 0.008105370216071606


In [71]:
df = pd.DataFrame(sp, columns=[
    'train_precision', 'test_precision', 'no_components', 'loss', 
    'learning_rate','item_alpha', 'user_alpha'])
df.head(3)

,train_precision,test_precision,no_components,loss,learning_rate,item_alpha,user_alpha
0,0.4198,0.0017,5,warp,0.01,0.05,0.05
1,0.3847,0.0023,5,warp,0.01,0.05,0.10
2,0.4196,0.0026,5,warp,0.01,0.05,0.15


In [118]:
pd.DataFrame(df.groupby('no_components')['test_precision'].mean())

,test_precision
no_components,
5,0.003539
10,0.003748
15,0.003731


In [57]:
pd.DataFrame(df.groupby('loss')['test_precision'].mean())

,test_precision
loss,
bpr,0.004231
warp,0.003115


In [58]:
pd.DataFrame(df.groupby('learning_rate')['test_precision'].mean())

,test_precision
learning_rate,
0.01,0.003137
0.05,0.003926
0.10,0.003956


In [59]:
pd.DataFrame(df.groupby('item_alpha')['test_precision'].mean())

,test_precision
item_alpha,
0.05,0.003656
0.10,0.003746
0.15,0.003617


In [60]:
pd.DataFrame(df.groupby('user_alpha')['test_precision'].mean())

,test_precision
user_alpha,
0.05,0.003772
0.10,0.003706
0.15,0.003541


Вывод:
* Выбор loss "bpr" приводит к увеличению метрики.
* Малый шаг обучения "learning_rate" = 0.01, так и малое число компонент "no_components" = 5 полюзу не дадут.
* Влияние изменения параметров смещений "item_alpha" и "user_alpha" не выявлено.

### *Отбор признаков* * 

Все данные категориальные, при ohe кодировании для товаров признаков становится невероятно много.      
Какие стратегии отбора признаков в классическом ML Вы знаете? Применимы ли они тут?  

Попробйте какие-нибудь стратегии. Удалось ли улучшить качество?

 \* *задание необязательно*
